# Data Cleaning Notebook


## TODO: 
    * Work on aggregating the title.akas DF. Each movie has multiple rows so we need to determine how we want to 
      merge this into our main one as each movie will have multiple entries 
    * Work on merging the list of actors into our list. Need to get a list of the actors for a given movie, merge it, 
      then one-hot encode it. 
    * Figure out how to featurize the movie title, maybe NLP vector would be the best but thats somewhat complicated 
      Could do things like length of the title, whether it contains nouns, etc. 

This notebook contains the relevant code for aggregating our CSV's into a singular one that can then be used in our models. 

Contains the One-Hot encoding, and featurization of the columns. 

In [88]:
import os
import pandas as pd 
import numpy as np 

#Descriptions of sets available at https://www.imdb.com/interfaces/

In [89]:
# Loading the Sets: 
sets = {} 

for file in os.listdir('DataSets'):
    file = file.replace('.tsv', '') 
    sets[file] = pd.read_csv(f"DataSets/{file}.tsv", sep='\t')
    if 'tconst' in sets[file].keys():
        sets[file].set_index('tconst',inplace=True)
    elif 'titleId' in sets[file].keys():
        sets[file].set_index('titleId', inplace=True) 
    else:
        print("Cant set index for ", file)
    print(f"Loaded {file}")
print("Loaded all datasets") 

Loaded title.ratings
Loaded title.principals
Loaded title.akas
Cant set index for  name.basics
Loaded name.basics
Loaded title.basics
Loaded title.crew
Loaded all datasets


# What we need from each dataset: 
### title.akas 
    * Language
    * region 
### title.basics
    * titleType -> Used to only keep movies, used in filtering. We should filter before we split, etc. 
    * primaryTitle -> We only care about the most popular title so use this one 
    * isAdult -> Why not include these 
    * startYear -> Year movie released, good to see to potentially capture trends of a timeperiod 
    * runtimeMinutes -> Good to know for how long a movie is 
    * genres -> One hot encoding on the genres it has 
### title.crew 
    * directors -> One hot encoding on top 100 directors 
    * writers -> One hot encoding on top 100 writers 
### title.episode 
    * We dont care about episodes so skip this one 
### title.principals  - Info about cast / crew for titles 
    * nconst -> Useful for determining which actor is who. Can use this with one-hot encoding for each movie to 
                determine top 100 actors and whether or not they were in a movie or not 
### title.ratings 
    * averageRating -> Weight average of all individual ratings, used as our target variable 
    * numVotes -> The number of votes it received - useful to somehow include this in our target, would want to 
                  weight training samples with move votes w/ more importance, can be used with models that allow that 
### name.basics
    * Can potentially include this later on in our featurizations if we need info about the people, currently I think 
      just their unique id from title.principals should be more than enough to capture actors. 

# Data Cleaning: 
    1) First get ids of all entries that are movies (So we are not including things that arent movies) 
    
    2) Then filter the raitings CSV to only include just movies, then move to actually just only keep the first X number of movies with a certain raitings count. 

In [90]:
print("DFs we have loaded: ", sets.keys())

DFs we have loaded:  dict_keys(['title.ratings', 'title.principals', 'title.akas', 'name.basics', 'title.basics', 'title.crew'])


In [91]:
## FILTERING SO WE ONLY HAVE MOVIES: 
df = sets['title.basics']

# Keep only the rows that have a titleType of movie
df = df[df['titleType'] == 'movie']

# The row ids that are just movies 
movie_ids = list(df.index)
#first 10 just to make sure its ids 
print(movie_ids[:10])


['tt0000502', 'tt0000574', 'tt0000591', 'tt0000615', 'tt0000630', 'tt0000675', 'tt0000679', 'tt0000739', 'tt0000793', 'tt0000814']


In [110]:
# Shifting over to the title.ratings CSV to get the first X number of movies with the most reviews 
df = sets['title.ratings']

#Filtering the df to only include the ids that were explicitly movies 
df = df.loc[movie_ids]

# Sorting the df by numVotes so its from most number of reviews -> less 
df.sort_values(by='numVotes', inplace=True, ascending=False)
# I dont know why I hardcoded these but checks 100k to 20k in batches of 10k  
for splitter in [100000, 90000, 80000, 70000, 60000, 50000, 40000, 30000, 20000]:
    top = list(df['numVotes'][:splitter])
    print(f"Total number of movies: {splitter} : Lowest raiting count: {top[-1]}")


# We will go with 30k for now to keep it simple :) 
TOTAL_MOVIES_TO_KEEP = 30000 

Total number of movies: 100000 : Lowest raiting count: 1481
Total number of movies: 90000 : Lowest raiting count: 1481
Total number of movies: 80000 : Lowest raiting count: 1481
Total number of movies: 70000 : Lowest raiting count: 1481
Total number of movies: 60000 : Lowest raiting count: 1481
Total number of movies: 50000 : Lowest raiting count: 1481
Total number of movies: 40000 : Lowest raiting count: 1481
Total number of movies: 30000 : Lowest raiting count: 1481
Total number of movies: 20000 : Lowest raiting count: 2996


In [111]:
# The ids that meet the criteria we are utilizing now in our main df. 
movie_ids = list(df['numVotes'][:TOTAL_MOVIES_TO_KEEP].index)
# Filter the DataFrame to only keep those ids :) 
print(df.shape)
df = df.loc[movie_ids]
print(df.shape)

# Our output final df, named final_df for ease. Contains the top 30k movies for the number of votes. 
final_df = df 

(29994, 2)
(29994, 2)


In [112]:
#IGNORING AKA RIGHT NOW AS IT HAS DUPLICATES SO WE NEED TO FIGURE OUT WHETHER TO AGGREGATE OR NOT

# If we ever need to change the columns we are merging into it, do so here :) 
AKA_COLS = ['region', 'language'] 
BASICS_COLS = ['isAdult', 'startYear', 'runtimeMinutes', 'genres']

# Grab the respective dfs, only grab the rows that are our movie ids we have chosen to work with 
df_aka = sets['title.akas'].loc[movie_ids]
df_basic = sets['title.basics'].loc[movie_ids]

/Users/ryankennedy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


In [113]:
df_aka

,ordering,title,region,language,types,attributes,isOriginalTitle
titleId,,,,,,,
tt0111161,10.0,The Shawshank Redemption,AU,\N,imdbDisplay,\N,0
tt0111161,11.0,Rita Hayworth and Shawshank Redemption,US,\N,working,\N,0
tt0111161,12.0,Rastegari dar Shawshank,IR,fa,imdbDisplay,\N,0
tt0111161,13.0,Les Évadés,FR,\N,imdbDisplay,\N,0
tt0111161,14.0,Avain pakoon,FI,\N,alternative,\N,0
...,...,...,...,...,...,...,...
tt0062207,5.0,O célebre roubo de Glasgow,PT,\N,imdbDisplay,\N,0
tt0062207,6.0,Manden bag kuppet,DK,\N,\N,\N,0
tt0062207,7.0,Rånet,SE,\N,\N,\N,0


In [114]:
# Filtering the dfs to only be the columns we are actually interested in 
df_aka = df_aka[AKA_COLS]
df_basic = df_basic[BASICS_COLS]

# Merging AKA and Basic  - ignoring right now 
# merged = df_aka.join(df_basic)

# Joining the merged above df into our final df 
# final_df = final_df.join(merged) 

final_df = final_df.join(df_basic) 

final_df

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres
tconst,,,,,,
tt0111161,9.3,2569002,0,1994,142,Drama
tt0468569,9.1,2534021,0,2008,152,"Action,Crime,Drama"
tt1375666,8.8,2254701,0,2010,148,"Action,Adventure,Sci-Fi"
tt0137523,8.8,2021525,0,1999,139,Drama
tt0109830,8.8,1981606,0,1994,142,"Drama,Romance"
...,...,...,...,...,...,...
tt13074322,5.5,1481,0,2020,85,Sci-Fi
tt1674775,5.4,1481,0,2011,94,"Drama,Thriller"
tt0054880,4.6,1481,0,1963,45,"Comedy,Drama,Horror"


In [115]:
CREW_COLS = ['directors','writers']
# Merging The Crew df into our main Df: Should probably do one hot encoding here before we actually merge it but idk 
crew = sets['title.crew'].loc[movie_ids]
crew = crew[CREW_COLS]

final_df = final_df.join(crew)


In [116]:
final_df

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,directors,writers
tconst,,,,,,,,
tt0111161,9.3,2569002,0,1994,142,Drama,nm0001104,"nm0000175,nm0001104"
tt0468569,9.1,2534021,0,2008,152,"Action,Crime,Drama",nm0634240,"nm0634300,nm0634240,nm0275286,nm0004170"
tt1375666,8.8,2254701,0,2010,148,"Action,Adventure,Sci-Fi",nm0634240,nm0634240
tt0137523,8.8,2021525,0,1999,139,Drama,nm0000399,"nm0657333,nm0880243"
tt0109830,8.8,1981606,0,1994,142,"Drama,Romance",nm0000709,"nm0343165,nm0744839"
...,...,...,...,...,...,...,...,...
tt13074322,5.5,1481,0,2020,85,Sci-Fi,nm8585595,nm8585595
tt1674775,5.4,1481,0,2011,94,"Drama,Thriller",nm0534665,"nm0866014,nm0534665"
tt0054880,4.6,1481,0,1963,45,"Comedy,Drama,Horror",nm0808542,nm0808542


In [117]:
# Grabs the top X number of categories, defaulted to 100. Utilized by our one-hot encoding system to
# determine what the most used number of categories are 
def top_categories(df, col_name, TOTAL_TO_KEEP = 100):
    """
        Provided a dataframe and the column name, returns the top 100 categories for this column 
        @param df: The DataFrame we are searching over 
        @param col_name: The column name we are examining the categories for 
        @param TOTAL_TO_KEEP: Constant specifying the total number of entries to keep. 
        :return A list of the top 100 categories for this column. 
    """
    frequencies = {}
    for entry in list(df[col_name]):
        # If we have a list of items iterate over each of them 
        if ',' in entry:
            for entry_sub in entry.split(','):
                if entry_sub in frequencies:
                    frequencies[entry_sub] += 1
                else:
                    frequencies[entry_sub] = 1 
        else:
            # Just a singular item so can compare it here directly. 
            if entry in frequencies:
                frequencies[entry] += 1
            else:
                frequencies[entry] = 1 
    
    lof_frequencies = []
    for key in frequencies.keys():\
        lof_frequencies.append( (key, frequencies[key]) )
    lof_frequencies.sort(key = (lambda pair: pair[1]))
        
    return [pair[0] for pair in lof_frequencies[:TOTAL_TO_KEEP]]


def encode_row(row, category):
    """
        Encodes a specific row / entry in our dataframe. If the row is a list, checks to see if the 
        category value exists in it, if it is not a list just checks to see if the row is equal to the category 
    """
    if ',' in row:
        return 1 if category in row.split(',') else 0 
    else:
        return 1 if category == row else 0 

def encode_non_categorical_row(row, lof_categories):
    """
        Encodes a row specifically looking to see if the value is not in our list of categories. If any of the 
        values does not exist in our list of categories return 1, else 0 
    """
    if ',' in row:
        for category in lof_categories:
            if category in row:
                return 1
        return 0 
    else: 
        return 1 if row not in lof_categories else 0 

# Converts a specific column to a one-hot encoding version of it.
def encode_column(df, col_name, TOTAL_TO_KEEP = 100):
    df = df.copy()
    """
        Provided a dataframe and column to encode, mutates a copy of the dataframe to have that a one-hot encoding of that 
        specific column. Will remove that specific column from the dataframe and replace it with TOTAL_TO_KEEP 
        columns for that value plus one more column to handle any categorical variable that was not in the top 
        TOTAL_TO_KEEP categories. 
        
        @param df: The dataframe we are mutating 
        @param col_name: The column we are encoding, this column is removed from the df and replaced with the encodings
        @param TOTAL_TO_KEEP: Number of categories we want to display, defaulted to 100 
        :return A Copy of the dataframe with the encoding 
    """
    categories = top_categories(df, col_name, TOTAL_TO_KEEP=TOTAL_TO_KEEP)
    
    # Our encoded columns for the dataset goes here 
    encoded_cols = {}
    
    lof_column = list(df[col_name])
    
    for category in categories: 
        encoded_column = [encode_row(row, category) for row in lof_column]
        
        encoded_col_name = f"{col_name}_{category}"
        df[encoded_col_name] = encoded_column
    # If any of the entries is not in our lof-categories featurize this under the non_100 category 
    df[f"{col_name}_non_100_category"] = [encode_row(row, categories) for row in lof_column]
    df.drop(columns=[col_name], inplace=True)
    return df 


In [118]:
# The purpose of this is to turn any string columns into integers so they aren't encoded
# TODO: Setting the value to 0 is incomplete. Find the mean first and set it after (this is difficult as it needs a numeric value before mean can be found)
final_df['startYear'] = final_df['startYear'].replace("\\N", np.NaN)
final_df['startYear'] = pd.to_numeric(final_df['startYear'])
final_df['startYear'] = final_df['startYear'].replace("\\N", final_df['startYear'].mean())

final_df['runtimeMinutes'] = final_df['runtimeMinutes'].replace("\\N", np.NaN)
final_df['runtimeMinutes'] = pd.to_numeric(final_df['runtimeMinutes'])
final_df['runtimeMinutes'] = final_df['runtimeMinutes'].replace(np.NaN, final_df['runtimeMinutes'].mean())

In [119]:
# TODO - Dont run one-hot encoding on the title / movie title, find different way to featurize this.
# COMMENTS ON WORK: I wanted to also try to encode regions but I noticed that pretty much most movies on this are available in most regions. They probably wouldn't really be useful features.
final_df

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,directors,writers
tconst,,,,,,,,
tt0111161,9.3,2569002,0,1994.0,142.0,Drama,nm0001104,"nm0000175,nm0001104"
tt0468569,9.1,2534021,0,2008.0,152.0,"Action,Crime,Drama",nm0634240,"nm0634300,nm0634240,nm0275286,nm0004170"
tt1375666,8.8,2254701,0,2010.0,148.0,"Action,Adventure,Sci-Fi",nm0634240,nm0634240
tt0137523,8.8,2021525,0,1999.0,139.0,Drama,nm0000399,"nm0657333,nm0880243"
tt0109830,8.8,1981606,0,1994.0,142.0,"Drama,Romance",nm0000709,"nm0343165,nm0744839"
...,...,...,...,...,...,...,...,...
tt13074322,5.5,1481,0,2020.0,85.0,Sci-Fi,nm8585595,nm8585595
tt1674775,5.4,1481,0,2011.0,94.0,"Drama,Thriller",nm0534665,"nm0866014,nm0534665"
tt0054880,4.6,1481,0,1963.0,45.0,"Comedy,Drama,Horror",nm0808542,nm0808542


In [120]:
### Encoding of title.principals to get the actors for a movie 
df = sets['title.principals']
# Filtering actors to just be the 
df = df.loc[movie_ids]

## We already encoded directors / writers so drop these from our table 
df = df.loc[(df['category'] != 'director') & (df['category'] != 'writer')]

# Grab the top 100 most common actors 
top_actors = top_categories(df, 'nconst')

df

,ordering,nconst,category,job,characters
tconst,,,,,
tt0111161,10,nm0290358,editor,\N,\N
tt0111161,1,nm0000209,actor,\N,"[""Andy Dufresne""]"
tt0111161,2,nm0000151,actor,\N,"[""Ellis Boyd 'Red' Redding""]"
tt0111161,3,nm0348409,actor,\N,"[""Warden Norton""]"
tt0111161,4,nm0006669,actor,\N,"[""Heywood""]"
...,...,...,...,...,...
tt0062207,1,nm0048939,actor,\N,"[""Paul Clifton""]"
tt0062207,2,nm0678620,actress,\N,"[""Kate Clifton""]"
tt0062207,3,nm0095718,actor,\N,"[""Inspector George Langdon""]"


In [121]:
# group = df.groupby('nconst')
# top = group.sum().sort_values(by='ordering', ascending=False)
# top_actors = list(top.index)[:100]

In [122]:
# Each key corresponds to a Movie, holds a dictionary of top actors and 1 / 0 designation for if it has that 
# Note, I thought it would be good to include the non_top_100 actors as just the total count they have instead of 1/0
# Can change this if it isnt accurate, but I thought it better captured the random actors. 
top_actor_encodings = {} 
# Pandas .loc is extremely expensive for such large tables, so its better to do this in one pass 
# Only doing the .loc once saves a ton of computation for each movie 
for movie_id in set(df.index):
    top_actor_encodings[movie_id] = {}
    lof_actors = df.loc[movie_id]
    sof_actors = set(lof_actors['nconst'])
    for actor in top_actors:
        if actor in sof_actors:
            top_actor_encodings[movie_id][actor] = 1 
        else:
            top_actor_encodings[movie_id][actor] = 0
    number_of_non_top = len(sof_actors.difference(set(top_actors)))
    top_actor_encodings[movie_id]['non_100'] = number_of_non_top

In [123]:
# Now go through and actually create the one-hot encoding columns 

# Some movies are missing data so we drop those 
movie_ids = set(movie_ids).intersection(set(df.index))
final_df = final_df.loc[movie_ids] 
print(final_df.shape)

# Adding the one-hot encoded columns 
for actor in top_actors + ['non_100']:
    column = []
    for movie_id in final_df.index:
        if movie_id not in top_actor_encodings:
            bad_ids.add()
        column.append(top_actor_encodings[movie_id][actor])
    col_name = f"actor_{actor}"
    final_df[col_name] = column


            

(29994, 8)


In [124]:
final_df

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,directors,writers,actor_nm0555550,actor_nm0245596,...,actor_nm1020124,actor_nm5831543,actor_nm0398469,actor_nm1331774,actor_nm3375122,actor_nm0641314,actor_nm0181425,actor_nm0263591,actor_nm6240141,actor_non_100
tconst,,,,,,,,,,,,,,,,,,,,,
tt0926063,5.8,11626,0,2008.0,87.0,"Action,Comedy,Horror",nm0084059,nm1298090,0,0,...,0,0,0,0,0,0,0,0,0,8
tt0110116,7.4,24628,0,1994.0,121.0,"Biography,Drama,Music",nm0741262,nm0741262,0,0,...,0,0,0,0,0,0,0,0,0,8
tt0083655,6.0,1898,0,1982.0,102.0,"Comedy,Family",nm0275269,"nm0274853,nm0549592,nm0646179,nm0869125",0,0,...,0,0,0,0,0,0,0,0,0,5
tt0405866,6.6,2744,0,2004.0,111.0,"Drama,Romance,Thriller",nm0001031,"nm0719334,nm1633128,nm0001031",0,0,...,0,0,0,0,0,0,0,0,0,7
tt5537140,5.6,4526,0,2016.0,132.0,"Crime,Drama,Mystery",nm0195264,nm0195264,0,0,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt0120118,6.3,3441,0,1996.0,93.0,"Drama,Family",nm0742428,"nm0623297,nm0742428",0,0,...,0,0,0,0,0,0,0,0,0,8
tt0116531,5.5,13343,0,1996.0,86.0,"Comedy,Crime,Romance",nm0000952,"nm0001878,nm0516598,nm0698493",0,0,...,0,0,0,0,0,0,0,0,0,6
tt0026138,7.8,47291,0,1935.0,75.0,"Drama,Horror,Sci-Fi",nm0001843,"nm0791217,nm0403398,nm0049721,nm0076653,nm0088...",0,0,...,0,0,0,0,0,0,0,0,0,4


In [125]:
# Finally, encode the following columns 
for col in ['genres', 'directors', 'writers']:
    final_df = encode_column(final_df, col)
final_df 

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,actor_nm0555550,actor_nm0245596,actor_nm0068501,actor_nm0001825,actor_nm0138287,...,writers_nm0244101,writers_nm0905942,writers_nm0630331,writers_nm0963380,writers_nm0307222,writers_nm4952714,writers_nm0777463,writers_nm1558799,writers_nm9946617,writers_non_100_category
tconst,,,,,,,,,,,,,,,,,,,,,
tt0926063,5.8,11626,0,2008.0,87.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0110116,7.4,24628,0,1994.0,121.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0083655,6.0,1898,0,1982.0,102.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0405866,6.6,2744,0,2004.0,111.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt5537140,5.6,4526,0,2016.0,132.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt0120118,6.3,3441,0,1996.0,93.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0116531,5.5,13343,0,1996.0,86.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0026138,7.8,47291,0,1935.0,75.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(29994, 334)


(29993, 334)

In [126]:
final_df.to_csv(f"merged.csv") 